<B> First load all data using sklearn Api <B>

In [3]:
from sklearn.datasets import fetch_20newsgroups as fetch_groups






In [129]:
Data = fetch_groups(subset ='all', categories= ['sci.med','comp.graphics'], shuffle=True, random_state=3116,return_X_y=True)

<b> defining the predictors and the target<b>
    

In [130]:
data = Data[0]
#print (data[1:100])
target = Data[1]
# remove stopping_words 
import nltk
from nltk.corpus import stopwords

In [180]:
print(target)

[0 1 0 ... 1 0 1]


<B> Here we extend the list of stop_words to include punc we use NLTK FOR THAT<b><br>


In [64]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

all_stopping_words = stopwords.words('english')
punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~|.'''
def split(punc): 
    return [char for char in punc]  
    #adding punctuation
punct = split(punc)     
all_stopping_words.extend(punct)


(1963, 32406)

In [55]:
print (all_stopping_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

<b> Here we loop over all the documents we have and remove the stop words <b><br>
1. we get every document then split if it contains a stop words we don't put it in the new text<br>
2. you will find the a atest document, the first one before and after under the next cell the after is under'-----'<br>

In [131]:
import re
def remove_stopping_words(text,stop_words):
    
    new_text = ' '.join([word for word in re.split('\W+', str(text).lower()) if word not in stop_words])
    return new_text


for i,document in enumerate(data):
        
        new_text= remove_stopping_words(document,all_stopping_words)
        data[i] = new_text
        if i ==0:
            print(document)
            print('------')
            print(new_text)
        
       

From: erikb@idt.unit.no (Erik Brenn)
Subject: graphics formats
Reply-To: erikb@idt.unit.no (Erik Brenn)
Organization: Norwegian Institue of Technology
Lines: 14

I'm currently looking for information about different graphics
formats, especially PPM, PCX BMP and perhaps GIF.
Does anyone know if there exist any files at some site
that describes these formats ???

Thanks !


-- 
          ~~~                       
         (o o)           | Erik Brenn ,email: erikb@idt.unit.no
        (  O  )   oOOO   | Faculty of Computer Science & Telematics
         \\_//    / /    | The Norwegian Institute of Technology, Trondheim
-oOOO--------------------| Not to make sense, just cents ! 

------
erikb idt unit erik brenn subject graphics formats reply erikb idt unit erik brenn organization norwegian institue technology lines 14 currently looking information different graphics formats especially ppm pcx bmp perhaps gif anyone know exist files site describes formats thanks erik brenn email erikb idt 

<B> Here we generate x_bag_of_words<B> <br>
1. we get all the data and join them <br>
2. we get all the unique data<br>
3. then we define the size where number of rows are the number of documents and columns len of the unique words<br>
4. each cell contains the number of occurences the column with index j appear in document i<br>
5. i use the count method for that <br>

In [139]:
import numpy as np
one_text = ' '.join(data)
one_text_splitted = one_text.split()
dict_unique_words = set(one_text_splitted)
X_bagWords = np.zeros([len(data),len(dict_unique_words)])
#print(X_bagWords.shape)
for i,document in enumerate(data) :
    for j,value in enumerate(dict_unique_words):
        if(value in document):
              X_bagWords[i][j] = document.count(value)  
                
    


<B> Here we start to generate  x_tf_idf<B> <br>
1. first we equate them with the bag_of words <br>
2. then we get the indices where values not equal to zero<br>
3. and for those values we divide the value by the corresponding length of the corresponding document<br>    

In [140]:
X_tf_idf = X_bagWords
indices_not_zero = np.where(X_tf_idf != 0)
    

In [159]:
def generating_X_tf_idf():
# putting the number of words in each document in the variable data_length
    data_length = [ len(document.split()) for document in data ]
# getting the i and j indices of the values not zero and dividing by the data length
    for index in range(len(indices_not_zero)):
        i = indices_not_zero[0][index]
        j = indices_not_zero[1][index]
        if(index == 0):
            print("this is i {} and this is j{}".format(i,j))
        X_tf_idf[i][j] = (X_tf_idf[i][j])/(data_length[i])
    return X_tf_idf

    
X_tf_idf = generating_X_tf_idf()        



this is i 0 and this is j11


<B> Here we start the process of splitting <B><br>
1. first of all  we split the indices by the target value


#the target value represents the category_id so 0 --> 'sci.med' and 1 -->'comp.graphics' 

In [169]:
import numpy as np
# get the indices of every value 
indices_sci_med = np.where(target == 0 )[0] # holds the indices where category is sci_med
indices_comp_graphics = np.where(target ==1)[0] # holds the indices where category is comp_graphics



<B> here we make syre that it is strasfied_sampling<B> <br>
1. we get the indices we got from above, 0.8 from the sci_med +0.8* comp_graphics go to training<br>
2. same on test and validation but 0.1 each

In [187]:
np.random.seed(3116)
train_indices_sci  = np.random.choice(indices_sci_med, int(0.8*len(indices_sci_med)),replace=False)
train_indices_comp = np.random.choice(indices_comp_graphics, int(0.8*len(indices_comp_graphics)),replace=False)
train_index  = np.concatenate((train_indices_sci,train_indices_comp))
# Rest indecx
Rest_indices_sci   = indices_sci_med[~np.isin(indices_sci_med,train_indices_sci)]
Rest_indices_comp  = indices_comp_graphics[~np.isin(indices_comp_graphics ,train_indices_comp)]
# vaidation index
#print (len(Rest_indices_sci))
validation_indices_sci  = np.random.choice(Rest_indices_sci, int(0.5*len(Rest_indices_sci)),replace=False)
test_index_sci = Rest_indices_sci[~np.isin(Rest_indices_sci,validation_indices_sci)]
validation_indices_comp = np.random.choice(Rest_indices_comp, int(0.5*len(Rest_indices_comp)),replace=False)
test_index_comp = Rest_indices_comp[~np.isin(Rest_indices_comp,validation_indices_comp)]
validation_index = np.concatenate((validation_indices_sci,validation_indices_comp))
test_index = np.concatenate((test_index_sci,test_index_comp))



<B> The length of the indices for training and validation and testing is under teh next cell<B>


In [188]:
print(len(train_index))
print(len(validation_index))
print (len(test_index))
#print(target[train_index] ==1)


1570
196
197


<B>Exercise 1 - Implementing Naive Bayes Classifier<B>

<B>Sepearte by class method<B><br>
1. based on the y_values it seperates the indices of the row of the dataframes <br>
2. one with zero and one with one<br>
3. it then return the dictionary of the indices of rows corresponding to every class<br>


In [314]:
def seperate_by_class( y_values):
    classes = {0:[],
               1:[]}
    for i in range(len(y_values)):
        if y_values[i] == 0:
           classes[0].append(i) 
        if y_values[i] == 1:
           classes[1].append(i)
    return classes        
        
    
    

<B>calculate_stastics_methods <B><br>
1. this method is for calcuating the mean and std of every column  <br>
2. it takes subset of dataframe bases on the indices passed<br>
3. calcualtes mean and std for every column and return them<br>

In [315]:
import pandas as pd
def calculate_stats(x,indices):
    stats={}
    #print(indices)
    x = pd.DataFrame(x)
    subset_x = x.iloc[indices, :]
    mean_not_f = subset_x.mean(axis = 0)
    std_not_f = subset_x.std(axis = 0)
    mean = []
    std = [ ]
    for i in range(len(mean_not_f)):
       mean.append(mean_not_f[i])
    for i in range (len(std_not_f)):
        std.append(std_not_f[i])
              
    return [mean,std]
        
    

<b> the naive_bayes_classfier method takes the predictors and the target <b><br>
    1. it gets the indices corrsponding of every class by calling seperate_by_class<br>
    2. then it goes and calcualte the statics for every column after they are seperated based on the class<br>
    3. it returns that dictionary<br>

In [316]:
def naive_bayes_classifer(x,y):
    # first thing is to sperate by class
    index_of_classes = seperate_by_class(y)
    #print(index_of_classes)
    # to calculate probablities of reaL_value we will use the gaussian_density function
    statsics_by_class = {}
    for key, values in index_of_classes.items() :
        statsics_by_class[key] = calculate_stats(x,values)
    return statsics_by_class    
    
    

<b> probablity_class method calculate probablity bu gaussian probality density function formula </b><br>
1. it gets the mean and std of that specfic column we are calcuating the probablity on <br>
2. and plug it in the equation <br>
3.return thr probablity <b>

In [317]:
import math
def probablity_class(class_value, column_value, index_of_column,stats):
    x = column_value
    mean = stats[class_value][0][index_of_column]
    std =  stats[class_value][1][index_of_column]
    std_squared = std**2
    # calculate gaussian probablity
    second_variable = math.exp( (-1*(x-mean)**2)/(2*std_squared+0.1))  
    first_variable = 1/(math.sqrt(2*math.pi*std_squared+0.1))
    return first_variable * second_variable
    
    
    
    

<b>predict_class method takes a row of the test_rows <b><br>
1. calcultes the probality for zero and one and returns the class of the bigger one<br>
2. the probality is accumaltive based on the conditional indeppendance we get the probality that class is
zero and x1= something * pprobality that x2 =something and class 0 and so on till we finish the row<br>

In [318]:
def predict_class(row,stats,p_zero,p_one):
    #print(row)
    probablity_zero =  p_zero
    probablity_one = p_one
    for i,column_value in enumerate(row):
        probablity_zero *= probablity_class(0,column_value,i,stats) 
    #print("probality_zero is", probablity_zero)
    for j,column_value in enumerate(row):
        probablity_one  *= probablity_class(1,column_value,j,stats)
    #print("probality_one is", probablity_one)
    if(probablity_zero <= probablity_one ):
        return 1
    else:
       return 0
        

<b>The predict method<b><br>
  1.calls predict_class on every row in the test dataset and return the presdictions and append it in an array<br>
  2. return that array<br>

In [301]:
def predict (X,stats,p_zero,p_one):
    predictions = []
    for i,row in enumerate(X):
        predictions.append(predict_class(row,stats,p_zero,p_one))
    return predictions     
    

<b> the test method was used to apply all the methods on small datsets and make sure that statiscs is corrct</b><br>
1. the statsics were coreect u can see them under the next cell<br>

In [308]:
def test() :
    x=[[3,2],[1,3],[2,4],[1,7],[9,19]]
    y =[1,0,0,0,1]
    x_test =[[3.393533211,2.331273381]]
    stats = naive_bayes_classifer(x,y)
    print("the mean  of class[0] is :{}, the std of class[0] is :{}".format(stats[0][0],stats[0][1]))
    print (" the mean of class[1] is :{}, the std of class[1] is:{}".format(stats[1][0],stats[1][1]))
    zero = np.where(y == 0)
    one = np.where (y==1)
    p_zero = len(zero)/len(y) 
    p_one  = len(one)/len(y)
    print("the predictions are :",predict(x_test,stats,p_zero,p_one))
#test()    

{0: [1, 2, 3], 1: [0, 4]}
[1, 2, 3]
[0, 4]
the mean  of class[0] is :[1.3333333333333333, 4.666666666666667], the std of class[0] is :[0.5773502691896257, 2.081665999466133]
 the mean of class[1] is :[6.0, 10.5], the std of class[1] is:[4.242640687119285, 12.020815280171307]
[3.393533211, 2.331273381]
probality_zero is 5.464394352680922e-05
probality_one is 0.0004102921642825742
the predictions are : [1]


<b> Executing_naive_bayes_method </b>
1. we get the predictions on the bag_of_words and the predictions on the tf-idf<br>
2. we return them in an aray

In [320]:
def executing_naive_bayes():
    stats_bag_words = naive_bayes_classifer(X_bagWords[train_index],target[train_index])
    zero = np.where(target[train_index]== 0)
    one = np.where (target[train_index]== 1)
    p_zero = len(zero)/len(target[train_index]) 
    p_one  = len(one)/len(target[train_index])
    predictions_bag_words = predict(X_bagWords[test_index],stats_bag_words,  p_zero,p_one )
    
    #accuracy_bag_words = calc_acc(predictions_bag_words,target[test_index])
    stats_tf_idf  = naive_bayes_classifer(X_tf_idf[train_index],target[train_index])
    predictions_tf_idf = predict(X_tf_idf[test_index],stats_tf_idf,p_zero,p_one )
    
    #accuracy_tf_idf = calc_acc(predictions_tf_idf,target[test_index])
    return [predictions_bag_words,predictions_tf_idf]

predictions = executing_naive_bayes()    

<b> Here we calculate the accuracy of the predictions<b><br>
1. once on the predictions of the bag_of_words<br>
2. the other on tf-idf<br>
2. the results are printed under the next cell<br>



In [328]:
def accuracy(predictions):
    error =0
    for i in range(len(target[test_index])):
        if predictions[i] != target[test_index][i]:
            error = error +1
        return error/len(target[test_index])
predictions_bag_of_words =  predictions[0]
predictions_tf_idf =  predictions[1]
error_percent_bag = accuracy(predictions_bag_of_words)
print("The error percentage is bag_of_words is:",error_percent_bag)
error_percent_tf_idf = accuracy(predictions_tf_idf)
print("The error percentage is tf-idf is: ",error_percent_tf_idf)

The error percentage is bag_of_words is: 0.005076142131979695
The error percentage is tf-idf is:  0.005076142131979695


<B>Exercise 2-  Implementing SVM Classifier via Scikit-Learn <B>

<B> The approach using the SVM classfier<B><br>
1. executing_svm method takes a mode paramter where the mode can be bag_of_words or tf-idf<br>
2. define paramter_space for grid_Search_o operate on<br>
3. we take the best_paramters and pass them to the model<br>
4. we fit in the training <br>
5. predict using the predict method<br>
6. calcualte teh accuracy using accuaracy_score<br>
7. the method is then called once on the bag_of_words and once on tf-idf<br>
8 the test_Scores are printed under the next cell<br>


In [209]:

from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
def executing_svm(mode):
    paramter_space = {
    'kernel':['linear', 'poly', 'rbf', 'sigmoid'],
     'C': [0.1,0.001,0.01,0.0001],
     'gamma' :['scale', 'auto']    
    }
    grid = GridSearchCV(SVC(),param_grid = paramter_space)
    svm_classfier = SVC()
    if(mode == 'bag_of_words'):
        grid.fit(X_bagWords[validation_index],target[validation_index])
        svm_classfier.set_params(**grid.best_params_)
        svm_classfier.fit(X_bagWords[train_index],target[train_index])
        predictions = svm_classfier.predict(X_bagWords[test_index])
        score = accuracy_score(target[test_index],predictions)
        print("The test accuracy on bag_of_words : {}".format(score))
    elif(mode == 'tf-idf'):
        grid.fit(X_bagWords[validation_index],target[validation_index])
        svm_classfier.set_params(**grid.best_params_)
        svm_classfier.fit(X_tf_idf[train_index],target[train_index])
        predictions = svm_classfier.predict(X_tf_idf[test_index])
        score = accuracy_score(target[test_index],predictions)   
        print("The test accuracy on the tf-idf : {}".format(score))
    #print("The predictions are:{}".format(predictions))
    #print ("The correct value are:{}".format(target[test_index]))
    
        
        



In [210]:
def call_svm():
    executing_svm('bag_of_words')
    executing_svm('tf-idf')
    
call_svm()    
    

The test accuracy on bag_of_words : 0.949238578680203
The test accuracy on the tf-idf : 0.949238578680203
